Ecological Tipping Points using TD3

In [2]:
import numpy as np
import stable_baselines3 as sb3
import gym
import gym_conservation
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
ALGOS = {"TD3": sb3.TD3, "PPO": sb3.PPO}


In [3]:
algo = "PPO"
RL = ALGOS[algo]
env_id = "conservation-v6"
team = "team_cboettig"
agent_id = env_id + "-" + algo + "-" + team
agent_id

'conservation-v6-PPO-team_cboettig'

In [4]:
seed = 24
np.random.seed(seed)

In [5]:
env = gym.make(env_id)

In [5]:

model = RL(
    "MlpPolicy",
    env,
    verbose=0,
    seed=seed,
    tensorboard_log="/var/log/tensorboard/conservation-v6"
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
if not os.path.exists((agent_id+".zip")):
    model.learn(total_timesteps=3000000)
    model.save(agent_id)

Logging to /var/log/tensorboard/conservation-v6/PPO_1
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 501       |
|    ep_rew_mean     | -1.49e+03 |
| time/              |           |
|    fps             | 728       |
|    iterations      | 1         |
|    time_elapsed    | 2         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 501         |
|    ep_rew_mean          | -1.34e+03   |
| time/                   |             |
|    fps                  | 560         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016986057 |
|    clip_fraction        | 0.166       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    exp

In [6]:
model = RL.load(agent_id)

score = evaluate_policy(model, Monitor(env), n_eval_episodes=10)
score

(46.8158493, 4.587082736626777)

In [7]:
sims = env.simulate(model, reps=5)
policy = env.policyfn(model, reps=5)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

rep0 = sims.query("rep < 5")
fig = px.line(rep0, x="time", y="state", color = "rep", title='5 example simulations')

#df = rep0.melt(id_vars=['time','rep'],var_name='metric', value_name='values')
#fig = px.line(df, x="time", y="values", color="metric", title='rep 0')

fig.show()

In [ ]:
df = policy.query("rep < 5")
fig = px.line(df, x="state", y="action", color="rep", title='policy (5 reps)')
fig.show()